# Домашнее задание по ML №1

## Предобработка

> **Задание**: Проведите предобработку текстов: если считаете нужным, выполните токенизацию, приведение к нижнему регистру, лемматизацию и/или стемминг. 
Ответьте на следующие вопросы:
1. Есть ли корреляция между средней длинной текста за день и DJIA?
2. Есть ли корреляция между количеством упоминаний Барака Обамы и США в день и DJIA? Учтите разные варианты написания США.
3. Каких статей больше: статей о России и Путине или об Исламском государстве (запрещенной законом РФ террористическая организации)?
4. О каких кризисах (crisis) пишут статьи?


Импортируем библиотеки. Для успешной работы с данными должны быть установлены следующие библиотеки:
* numpy
* pandas
* sklearn
* matplotlib
* nltk

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

Загружаем данные, смотрим на них:

In [5]:
df_news = pd.read_csv('Combined_News_DJIA.csv')

In [6]:
df_news.head()

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,...,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"b""Georgia 'downs two Russian warplanes' as cou...",b'BREAKING: Musharraf to be impeached.',b'Russia Today: Columns of troops roll into So...,b'Russian tanks are moving towards the capital...,"b""Afghan children raped with 'impunity,' U.N. ...",b'150 Russian tanks have entered South Ossetia...,"b""Breaking: Georgia invades South Ossetia, Rus...","b""The 'enemy combatent' trials are nothing but...",...,b'Georgia Invades South Ossetia - if Russia ge...,b'Al-Qaeda Faces Islamist Backlash',"b'Condoleezza Rice: ""The US would not act to p...",b'This is a busy day: The European Union has ...,"b""Georgia will withdraw 1,000 soldiers from Ir...",b'Why the Pentagon Thinks Attacking Iran is a ...,b'Caucasus in crisis: Georgia invades South Os...,b'Indian shoe manufactory - And again in a se...,b'Visitors Suffering from Mental Illnesses Ban...,"b""No Help for Mexico's Kidnapping Surge"""
1,2008-08-11,1,b'Why wont America and Nato help us? If they w...,b'Bush puts foot down on Georgian conflict',"b""Jewish Georgian minister: Thanks to Israeli ...",b'Georgian army flees in disarray as Russians ...,"b""Olympic opening ceremony fireworks 'faked'""",b'What were the Mossad with fraudulent New Zea...,b'Russia angered by Israeli military sale to G...,b'An American citizen living in S.Ossetia blam...,...,b'Israel and the US behind the Georgian aggres...,"b'""Do not believe TV, neither Russian nor Geor...",b'Riots are still going on in Montreal (Canada...,b'China to overtake US as largest manufacturer',b'War in South Ossetia [PICS]',b'Israeli Physicians Group Condemns State Tort...,b' Russia has just beaten the United States ov...,b'Perhaps *the* question about the Georgia - R...,b'Russia is so much better at war',"b""So this is what it's come to: trading sex fo..."
2,2008-08-12,0,b'Remember that adorable 9-year-old who sang a...,"b""Russia 'ends Georgia operation'""","b'""If we had no sexual harassment we would hav...","b""Al-Qa'eda is losing support in Iraq because ...",b'Ceasefire in Georgia: Putin Outmaneuvers the...,b'Why Microsoft and Intel tried to kill the XO...,b'Stratfor: The Russo-Georgian War and the Bal...,"b""I'm Trying to Get a Sense of This Whole Geor...",...,b'U.S. troops still in Georgia (did you know t...,b'Why Russias response to Georgia was right',"b'Gorbachev accuses U.S. of making a ""serious ...","b'Russia, Georgia, and NATO: Cold War Two'",b'Remember that adorable 62-year-old who led y...,b'War in Georgia: The Israeli connection',b'All signs point to the US encouraging Georgi...,b'Christopher King argues that the US and NATO...,b'America: The New Mexico?',"b""BBC NEWS | Asia-Pacific | Extinction 'by man..."
3,2008-08-13,0,b' U.S. refuses Israel weapons to attack Iran:...,"b""When the president ordered to attack Tskhinv...",b' Israel clears troops who killed Reuters cam...,b'Britain\'s policy of being tough on drugs is...,b'Body of 14 year old found in trunk; Latest (...,b'China has moved 10 *million* quake survivors...,"b""Bush announces Operation Get All Up In Russi...",b'Russian forces sink Georgian ships ',...,b'Elephants extinct by 2020?',b'US humanitarian missions soon in Georgia - i...,"b""Georgia's DDOS came from US sources""","b'Russian convoy heads into Georgia, violating...",b'Israeli defence minister: US against strike ...,b'Gorbachev: We Had No Choice',b'Witness: Russian forces head towards Tbilisi...,b' Quarter of Russians blame U.S. for conflict...,b'Georgian president says US military will ta...,b'2006: Nobel laureate Aleksander Solzhenitsyn...
4,2008-08-14,1,b'All the experts admit that we should legalis...,b'War in South Osetia - 89 pictures made by a ...,b'Swedish wrestler Ara Abrahamian throws away ...,b'Russia exaggerated the death toll in South O...,b'Missile That Killed 9 Inside Pakistan May Ha...,"b""Rushdie Condemns Random House's Refusal to P...",b'Poland and US agree 

Стоп-слова:

In [8]:
from nltk.corpus import stopwords
stopset = stopwords.words('english')
stopset[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your']

Мы не будем лемматизировать заголовки новостей, потому что <s>этот лемматизатор в nltk как мёртвому припарка</s> <s>чувак-победитель на kaggle ничего не лемматизировал</s> информация о словоизменении может оказаться полезной. 

In [9]:
CountVectorizer()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [24]:
from nltk.tokenize import word_tokenize
word_tokenize('BREAKING: Musharraf to be impeached')

['BREAKING', ':', 'Musharraf', 'to', 'be', 'impeached']

## Обучение классификаторов

> **Задание**: Вам предстоит решить следующую задачу: по текстам новостей за день определить, вырастет или понизится DJIA. То есть, метки класса (y) заданы DJIA, признаки (X) требуется извлечь из текстов.

> Обучающее и тестовое множество строится так: данные до начала 2015 года используются для обучения, данные с 2015 года и позже – для тестирования.

> Используйте любой известный вам алгоритм классификации текстов для того,  Используйте $tf-idf$ преобразование, сингулярное разложение, нормировку признакого пространства и любые другие техники обработки данных, которые вы считаете нужным. Используйте accuracy и F-measure  для оценки качества классификации. Покажите, как  $tf-idf$ преобразование или сингулярное разложение или любая другая использованная вами техника влияет на качество классификации. 

> Если у выбранного вами алгоритма есть гиперпараметры (например, alpha в преобразовании Лапласа для метода наивного Байеса), покажите, как изменение гиперпараметра влияет на качество классификации. 

Разделяем выборку на train и test, как сказано в задании:

In [22]:
train = df_news[df_news['Date'] < '2015-01-01']
test = df_news[df_news['Date'] > '2014-12-31']

In [21]:
from sklearn.pipeline import Pipeline

А теперь попробуем использовать лемматизатор <s>и понять, почему не надо этого делать</s>:

In [25]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

In [26]:
wnl.lemmatize('added')

'added'

In [27]:
wnl.lemmatize('kittens')

'kitten'

In [28]:
wnl.lemmatize('longer')

'longer'

In [ ]:
Как мы видим, лемматизируются только существитель